In [107]:
from itertools import count
import inspect


class ModelComponent(object):
    pass


class OrderedMixin(object):

    _ids = count(0)

    def __init__(self, *args, **kwargs):
        self._order = next(self._ids)
        super().__init__(*args, **kwargs)

    @property
    def order(self):
        try:
            return self._order
        except AttributeError:
            return -1


class Method(ModelComponent):

    def __init__(self, func: callable=None, label: str=None, *args, **kwargs) -> None:
        self.func = func
        super().__init__(*args, **kwargs)

    def __call__(self, *args, **params):
        # Check if being used as a decorator
        if len(args) == 1 and callable(args[0]) and self.func is None:
            self.func = args[0]
            return self
        # Assume being used as a function
        else:
            return self.analyze(**params)

    def analyze(self, **params):
        return self.func(**params)
    

class Schema(ModelComponent):

    def __init__(self, schema: str=None, *args, **kwargs):
        self.schema = schema

    def __call__(self, **params):
        self.analyze(**params)

    def analyze(self, **params):
        raise NotImplementedError("Subclasses must implement analyze method")


class Sequence(ModelComponent, OrderedMixin):

    def __init__(self):
        pass

    def __call__(self, *args, **params):
        # Check if being used as a decorator
        if len(args) == 1 and isinstance(args[0], type):
            self.func = args[0]
            return self
        # Assume being used as a function
        else:
            return self.analyze(**params)


    @property
    def members(self):
        return [m for m in self.__dir__ if issubclass(m, ModelComponent)]

    def analyze(self, **params):
        raise NotImplementedError("Subclasses must implement analyze method")

In [108]:
def ordered(order=None):
    def _decorator(cls):
        if not order is None:
            cls._order = order
        return cls
    return _decorator

def processor(*args, **kwargs):
    def _decorator(func):
        return Method(*args, **kwargs)
    return _decorator

In [119]:
def x():
    return Sequence

class Test(x()):
    pass

issubclass(Test, Sequence)

True

In [109]:
class Model(Sequence):

    @Container
    class Spfs(Sequence):
        order = 0

        @Method
        def test(a):
            return a
        
        @Container
        class SpfParams(Sequence):
            order = 0
            a = 1
            b = 2
            spf_params = Schema('spf.json')

        @Container
        class SpfCalculations(Sequence):
            order = 1
            
            @Method
            def n_kabco(a, b):
                return a + b

            @Method
            def n_kabc(a, b):
                return a + b

In [115]:
class SpfCalculations(Sequence):
    order = 1
    
    @Method
    def n_kabco(a, b):
        return a + b
    @Method
    def n_kabc(a, b):
        return a + b
    
m = SpfCalculations()
type(m.n_kabco)

__main__.Method

In [116]:
m.members

[]

In [111]:
m = Model()
m.members

[]

In [112]:
m.__dict__

{}